In [2]:
import numpy as np
import spacy
import csv
from collections import defaultdict

In [3]:
def glove_to_dict(glove_file):
    glove_dict = defaultdict(lambda: np.zeros(300))
    
    with open(glove_file) as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quoting=csv.QUOTE_NONE)
        for row in reader:
            values = np.array(row[1:])
            glove_dict[row[0]] = values.astype(np.float)
    return glove_dict

In [4]:
def preprocess(line, glove_dict):
    output_list = []
    line2 = line.replace('<br />', ' ')
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(line2)
    for token in doc:
        output_list.append(glove_dict[token.lower_])
    return np.array(output_list)

In [16]:
def dataloader(csv_file, glove_dict):
    x_train, y_train, x_test, y_test = [], [], [], []
    count = 0
    
    with open(csv_file, encoding="latin-1") as csvFile:
        reader = csv.reader(csvFile)
        
        for row in reader:
            if row[1] == 'train':
                if row[3] == 'neg':
                    y_train.append(np.array([1, 0]))
                elif row[3] == 'pos':
                    y_train.append(np.array([0, 1]))
                else:
                    continue
                x_train.append(preprocess(row[2], glove_dict))
                
            elif row[1] == 'test':
                if row[3] == 'neg':
                    y_test.append(np.array([1, 0]))
                elif row[3] == 'pos':
                    y_test.append(np.array([0, 1]))
                else:
                    continue
                x_test.append(preprocess(row[2], glove_dict))
            count += 1
            if count % 100 == 0:
                print(count)
    
    return x_train, y_train, x_test, y_test

In [6]:
glove_dict = glove_to_dict('/mnt/hdd/datasets/glove_embeddings/glove.840B.300d.txt')

In [17]:
x_train, y_train, x_test, y_test = dataloader('imdb_master.csv', glove_dict)

KeyboardInterrupt: 

In [ ]:
pickle.dump(x_train, open("x_train.p", "wb"))
pickle.dump(y_train, open("y_train.p", "wb"))
pickle.dump(x_test, open("x_test.p", "wb"))
pickle.dump(y_test, open("y_test.p", "wb"))